In [1]:

import torch
import yaml
import numpy as np

from prithvi_100m.Prithvi import MaskedAutoencoderViT
from dataset import XBDDataset
from model import DamageSegmentation
from model_compiler import ModelCompiler
from config import MODEL_ARGS, CLASS_MAPPING

/Users/michaelskaug/venv/prithvi-change-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model

### Pre-trained encoder

In [2]:
# load weights
weights_path = './prithvi_100m/Prithvi_100M.pt'
checkpoint = torch.load(weights_path, map_location='cpu')

# Drop the weights associated with the three IR channels which don't exist in this dataset
checkpoint['patch_embed.proj.weight'] = checkpoint['patch_embed.proj.weight'][:,:3]

# read model config
model_cfg_path = './prithvi_100m/Prithvi_100M_config.yaml'
with open(model_cfg_path) as f:
    model_config = yaml.safe_load(f)

model_args = model_config['model_args']
train_args = model_config['train_params']

# 2 frames: pre image and post image
model_args['num_frames'] = 2

model_args['img_size'] = 1024

model_args['in_chans'] = 3

# instantiate model
encoder_model = MaskedAutoencoderViT(**model_args)
encoder_model.eval()

# load weights into model
del checkpoint['pos_embed']
del checkpoint['decoder_pos_embed']
del checkpoint['decoder_pred.weight']
del checkpoint['decoder_pred.bias']
encoder_model.load_state_dict(checkpoint, strict=False)

_IncompatibleKeys(missing_keys=['pos_embed', 'decoder_pos_embed', 'decoder_pred.weight', 'decoder_pred.bias'], unexpected_keys=[])

### Segmentation model

In [3]:
segmentation_model = DamageSegmentation(
    encoder_model.forward_encoder
)

In [5]:
compiled_model = ModelCompiler(
    segmentation_model,
    out_dir=MODEL_ARGS['out_dir'],
    num_classes=MODEL_ARGS['num_classes'],
    class_mapping=CLASS_MAPPING,
    model_init_type=MODEL_ARGS['model_init_type'],
)

----------No GPU available, using CPU instead----------
initialize network with kaiming.
total number of trainable parameters: 14.1M
---------- Vanilla Model compiled successfully ----------


# Data

In [6]:
train_ds = XBDDataset('data/geotiffs/hold')

train_data_loader = torch.utils.data.DataLoader(train_ds, batch_size=1, shuffle=True)

In [7]:
val_ds = XBDDataset('data/geotiffs/hold')

validation_data_loader = torch.utils.data.DataLoader(val_ds, batch_size=1, shuffle=False)

# Fit

In [8]:
loss_fn = torch.nn.CrossEntropyLoss(weight=torch.Tensor(MODEL_ARGS['loss_weights']))

In [ ]:
compiled_model.fit(
    train_data_loader,
    validation_data_loader, 
    epochs=MODEL_ARGS['epochs'],
    optimizer_name=MODEL_ARGS['optimizer'],
    lr_init=MODEL_ARGS['LR'],
    lr_policy=MODEL_ARGS['LR_policy'], 
    loss_fn=loss_fn,
    checkpoint_interval=MODEL_ARGS['checkpoint_interval'],
    resume=MODEL_ARGS['resume'],
    resume_epoch=MODEL_ARGS['resume_epoch'],
    **MODEL_ARGS['LR_kwargs']
)